# インポート

In [1]:

import numpy as np
import pandas as pd
import os
import pickle
import gc 

# 分布確認
import ydata_profiling as pdp

# 可視化
import matplotlib.pyplot as plt

# 前処理
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder

# バリデーション
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

# 評価指標
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

# モデリング: lightgbm
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

import sys

# matplotilbで日本語表示したい場合はこれをinstallしてインポートする
import japanize_matplotlib

from pathlib import Path

# Path().resolve()
# sys.path.append(os.path.join(Path().resolve(), '../source/'))




## データ確認

In [2]:

df_train = pd.read_csv("../data/input/train.csv") # 学習データ
df_test = pd.read_csv("../data/input/test.csv")   # テストデータ

# データセットの作成

* 性別は数値でないためOneHotVectorで数値化する

In [3]:
# 性別をベクトル化
df_one = pd.get_dummies(df_train[["Sex"]], dummy_na=False, drop_first=False)
df_one = df_one.astype(np.int64)

df_sex_test = pd.get_dummies(df_test[["Sex"]], dummy_na=False, drop_first=False)
df_sex_test = df_sex_test.astype(np.int64)


In [4]:
# データセット作成
x_train = pd.concat([df_one, df_train[["Pclass", "Fare"]]], axis=1)
y_train = df_train[["Survived"]]
id_train = df_train[["PassengerId"]]

x_test = pd.concat([df_sex_test, df_test[["Pclass", "Fare"]]], axis=1)

# 検証の方針

## 検証データ
* 交差検証を用いる


## モデル
* LightGBMを用いる

# ベースラインの作成

## ハイパーパラメータの探索

In [5]:

# 自作関数インポート
from src import util
import importlib
importlib.reload(util)

# 最適化パラメータ探索
best_trial = util.optimize(util.objective_func, x_train, y_train)
print(best_trial.params)
print(util.params)

util.params.update(best_trial.params)
util.params

[I 2023-10-22 21:52:54,039] A new study created in memory with name: no-name-644fcb45-4975-4100-8403-e9271323ecf7


[LightGBM] [Warning] feature_fraction is set=0.7756573845414456, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7756573845414456
[LightGBM] [Warning] min_data_in_leaf is set=61, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=61
[LightGBM] [Warning] lambda_l1 is set=0.492522233779106, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.492522233779106
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=4.792414358623587e-05, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=4.792414358623587e-05
[LightGBM] [Warning] bagging_fraction is set=0.8597344848927815, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8597344848927815
[LightGBM] [Warning] lambda_l2 is set=83.76388146302445, reg_lambda=0.0 will be ignored. Current value: lambda_l2=83.76388146302445
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_f

[I 2023-10-22 21:52:54,405] Trial 0 finished with value: 0.7541711129244868 and parameters: {'num_leaves': 181, 'min_data_in_leaf': 61, 'min_sum_hessian_in_leaf': 4.792414358623587e-05, 'feature_fraction': 0.7756573845414456, 'bagging_fraction': 0.8597344848927815, 'lambda_l1': 0.492522233779106, 'lambda_l2': 83.76388146302445}. Best is trial 0 with value: 0.7541711129244868.


[LightGBM] [Warning] feature_fraction is set=0.7756573845414456, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7756573845414456
[LightGBM] [Warning] min_data_in_leaf is set=61, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=61
[LightGBM] [Warning] lambda_l1 is set=0.492522233779106, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.492522233779106
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=4.792414358623587e-05, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=4.792414358623587e-05
[LightGBM] [Warning] bagging_fraction is set=0.8597344848927815, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8597344848927815
[LightGBM] [Warning] lambda_l2 is set=83.76388146302445, reg_lambda=0.0 will be ignored. Current value: lambda_l2=83.76388146302445
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_f

[I 2023-10-22 21:52:54,715] Trial 1 finished with value: 0.7588600841127363 and parameters: {'num_leaves': 178, 'min_data_in_leaf': 99, 'min_sum_hessian_in_leaf': 0.00015009027543233888, 'feature_fraction': 0.6715890080754348, 'bagging_fraction': 0.8645248536920208, 'lambda_l1': 0.567922374174008, 'lambda_l2': 0.01732652966363563}. Best is trial 1 with value: 0.7588600841127363.


...

[I 2023-10-22 21:53:02,380] Trial 29 finished with value: 0.7867553825874083 and parameters: {'num_leaves': 76, 'min_data_in_leaf': 145, 'min_sum_hessian_in_leaf': 3.216329135917105e-05, 'feature_fraction': 0.6984804982745697, 'bagging_fraction': 0.6397548071515035, 'lambda_l1': 0.4224534932644461, 'lambda_l2': 16.64848046417389}. Best is trial 2 with value: 0.7867553825874083.


[LightGBM] [Warning] feature_fraction is set=0.6984804982745697, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6984804982745697
[LightGBM] [Warning] min_data_in_leaf is set=145, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=145
[LightGBM] [Warning] lambda_l1 is set=0.4224534932644461, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.4224534932644461
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=3.216329135917105e-05, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=3.216329135917105e-05
[LightGBM] [Warning] bagging_fraction is set=0.6397548071515035, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6397548071515035
[LightGBM] [Warning] lambda_l2 is set=16.64848046417389, reg_lambda=0.0 will be ignored. Current value: lambda_l2=16.64848046417389
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] featu

{'num_leaves': 107,
 'min_data_in_leaf': 149,
 'min_sum_hessian_in_leaf': 3.52756635172055e-05,
 'feature_fraction': 0.5877258780737462,
 'bagging_fraction': 0.7657756869209191,
 'lambda_l1': 1.3406343673102123,
 'lambda_l2': 3.4482904089131434}

{'num_leaves': 107, 'min_data_in_leaf': 149, 'min_sum_hessian_in_leaf': 3.52756635172055e-05, 'feature_fraction': 0.5877258780737462, 'bagging_fraction': 0.7657756869209191, 'lambda_l1': 1.3406343673102123, 'lambda_l2': 3.4482904089131434}
{'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.1, 'num_leaves': 16, 'n_estimators': 100000, 'random_state': 123, 'importance_type': 'gain'}


{'boosting_type': 'gbdt',
 'objective': 'binary',
 'metric': 'auc',
 'learning_rate': 0.1,
 'num_leaves': 107,
 'n_estimators': 100000,
 'random_state': 123,
 'importance_type': 'gain',
 'min_data_in_leaf': 149,
 'min_sum_hessian_in_leaf': 3.52756635172055e-05,
 'feature_fraction': 0.5877258780737462,
 'bagging_fraction': 0.7657756869209191,
 'lambda_l1': 1.3406343673102123,
 'lambda_l2': 3.4482904089131434}

### モデルの学習

In [6]:

# CV実行
imp, metrics, model_list = util.train_cv(x_train, y_train, id_train, util.params, n_splits=5)

-------------------- 0 --------------------
x_train (712, 4) y_valid (712, 1)
x_valid (179, 4) y_valid (179, 1)
y_train:0.384, y_tr:0.383, y_va:0.385
[LightGBM] [Warning] feature_fraction is set=0.5877258780737462, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5877258780737462
[LightGBM] [Warning] min_data_in_leaf is set=149, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=149
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=3.52756635172055e-05, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=3.52756635172055e-05
[LightGBM] [Warning] bagging_fraction is set=0.7657756869209191, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7657756869209191
[LightGBM] [Warning] lambda_l1 is set=1.3406343673102123, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.3406343673102123
[LightGBM] [Warning] lambda_l2 is set=3.4482904089131434, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.4

### 重要度の評価

In [7]:
imp.sort_values("imp", ascending=False, ignore_index=True)

,col,imp,imp_std
0,Sex_female,761.003072,46.443744
1,Sex_male,325.901290,24.337061
2,Pclass,260.195238,35.713838
3,Fare,71.360444,31.700849


# モデル推論

In [8]:
df_sex_test = pd.get_dummies(df_test[["Sex"]], dummy_na=False, drop_first=False)
df_sex_test = df_sex_test.astype(np.int64)
x_test = pd.concat([df_sex_test, df_test[["Pclass", "Fare"]]], axis=1)
id_test = df_test[["PassengerId"]]

In [9]:
# 結果を辞書に保存
solution = {}
 
# 各モデルで予測
for i, model in enumerate(model_list):
    test_pred = model.predict(x_test)
    solution[str(i) + "_model"] = test_pred

# 辞書からDataFrameに変更
solution = pd.DataFrame(solution)
solution

,0_model,1_model,2_model,3_model,4_model
0,0,0,0,0,0
1,1,1,1,1,1
2,0,0,0,0,0
3,0,0,0,0,0
4,1,1,1,1,1
...,...,...,...,...,...
413,0,0,0,0,0
414,1,1,1,1,1
415,0,0,0,0,0
416,0,0,0,0,0


In [10]:
# 多数決 (最頻値)を取得
solution_max = solution.mode(axis = 1).values

In [11]:
# PassengerIdを取得
PassengerId = np.array(df_test["PassengerId"]).astype(int)
 
# my_prediction(予測データ）とPassengerIdをデータフレームへ落とし込む
my_solution = pd.DataFrame(solution_max.astype(int), index = PassengerId, columns = ["Survived"])
 
# my_tree_one.csvとして書き出し
my_solution.to_csv("../data/output/002/submission.csv", index_label = ["PassengerId"])